In [ ]:
class Node:
    def __init__(self, index, demand, serv_time, time_window, lat, long):
        self.index = index
        self.demand = demand
        self.serv_time = serv_time
        self.time_window = time_window
        self.lat = lat
        self.long = long

    def __index__(self):
        return self.index

    def __hash__(self):
        return self.index

    def __repr__(self):
        return f"Customer <{self.index}>"

In [ ]:
from docplex.mp.model import Model

def build_VRPTW_problem(capacity, nodes, times, vehicles,**kwargs):
    
    #Parameters
    customers = [cstmrs for cstmrs in nodes if cstmrs.index != 0] 
    #Big M
    bigM = 1000

    #Model declaration
    m = Model(name = 'VRPTW', **kwargs)

    #----- Variables -----
    #Arc covered by vehicle
    m.x_var = m.binary_var_cube(vehicles,nodes,nodes, name= 'route')
    #Start time for the service of a customer
    m.s_var = m.continuous_var_matrix(vehicles, nodes, name = 'service_time')

    #----- Constraints -----
    #Customer satisfaction
    m.covering_cts = []
    for i in customers: 
        cust_satisf_ct = m.sum(
            m.x_var[u,i,j] for j in nodes for u in vehicles)>= 1
        cust_satisf_ct.name = 'customer_satisf_{0!s}'.format(i.index)
        m.covering_cts.append(cust_satisf_ct)

    m.add_constraints(m.covering_cts)
    
    #Route structure
    m.flow_cts = []
    for i in nodes:
        for u in vehicles:
            flow_ct = (m.sum(m.x_var[u,i,j] for j in nodes) - 
                m.sum(m.x_var[u,j,i] for j in nodes)) == 0
            flow_ct.name = 'flow_constraint_{0!s}_{1!s}'.format(i.index,u)
            m.flow_cts.append(flow_ct)

    for u in vehicles: 
        flow_ct = m.sum(m.x_var[u,nodes[0],i] for i in nodes) <= 1
        flow_ct.name = 'flow_constraint_{0!s}_{1!s}_origin'.format(nodes[0].index,u)
        m.flow_cts.append(flow_ct)

    m.add_constraints(m.flow_cts)

    #Vehicle capacity
    m.capacity_cts = []
    for u in vehicles:
        capacity_ct = m.sum(nodes[i].demand * m.x_var[u,i,j] for i in nodes for j in nodes) <= capacity
        capacity_ct.name = 'capacity_vehicle{0!s}'.format(u)
        m.capacity_cts.append(capacity_ct)
    
    m.add_constraints(m.capacity_cts)

    #Time windows
    m.time_window_cts = []
    for i in nodes:
        for j in customers: 
            for u in vehicles:
                time_window_ct = m.s_var[u,i] + i.serv_time + times[i.index][j.index] - m.s_var[u,j] + bigM*m.x_var[u,i,j] <= bigM
                time_window_ct.name = 'time_window_{0!s}_{1!s}_{2!s}'.format(u,i.index,j.index)
                m.time_window_cts.append(time_window_ct)
    
    for i in customers:
        for u in vehicles: 
            time_window_ct = m.s_var[u,i] + i.serv_time + times[i][0] - nodes[0].time_window[1] + bigM*m.x_var[u,i,nodes[0]] <= bigM
            time_window_ct.name = 'time_window_{0!s}_{1!s}_{2!s}_arrival'.format(u,i.index,nodes[0].index)
            m.time_window_cts.append(time_window_ct)

    for i in nodes:
        for u in vehicles:
            time_window_ct_lb = i.time_window[0] <= m.s_var[u,i]
            time_window_ct_lb.name = 'time_window_lb_{0!s}'.format(i.index)
            time_window_ct_ub = i.time_window[1] >= m.s_var[u,i]
            time_window_ct_ub.name = 'time_window_ub_{0!s}'.format(i.index)
            m.time_window_cts.append(time_window_ct_lb)
            m.time_window_cts.append(time_window_ct_ub)

    m.add_constraints(m.time_window_cts)

    #----- Objective Function -----
    #Minimize the total cost
    m.total_cost = m.sum(times[i][j]*m.x_var[u,i,j] for i in nodes for j in nodes for u in vehicles)
    m.minimize(m.total_cost)

    return m

In [ ]:
'''
To create a node we use the following parameters
index: int
    The index of the node in the directed graph for the network (The index 0 is used for the refinery)
demand: float
    The demand for each one of the nodes 
serv_time: float
    The service time for the nodes
time_window: list
    The time window where the node can be served
lat: float
    latitude of the node
long: float
    longitude of the node
'''
refinery = Node(0,0,0,[0,120], -11.920058,-77.129718)
gas_station_1 = Node(1,35,5,[15,80], -12.050424, -76.961938)
gas_station_2 = Node(2,25,6,[35,115], -12.139093, -77.017575)
gas_station_3 = Node(3,35,5,[12,68], -11.971129, -76.753334)
gas_station_4 = Node(4,45,6,[24,96], -12.056154, -77.057922)

vehicles = [0,1]
capacity = 100

nodes = [refinery, gas_station_1, gas_station_2, gas_station_3, gas_station_4]

In [ ]:
#isto é só as distâncias
times = [[round(((node_origin.lat - node_destin.lat)**2 + 
    (node_origin.long - node_destin.long)**2)**(1/2),3)*100 
    for node_destin in nodes] for node_origin in nodes]

In [ ]:
mod = build_VRPTW_problem(capacity, nodes, times, vehicles)

In [ ]:
import json
sol = mod.solve()
sol_dict = json.loads(sol.export_as_json_string())
sol_dict['CPLEXSolution']['variables'][0:6]

In [ ]:
v0 = [nodes[0], nodes[3], nodes[1],nodes[2],nodes[0]]
v1 = [nodes[0], nodes[4],nodes[0]]

v = [v0, v1]

routes = []

for vehicle in vehicles:
    for track in zip(v[vehicle],v[vehicle][1:] + v[vehicle][:1]):
        start = (track[0].lat, track[0].long)
        end = (track[1].lat, track[1].long)
        start_node = ox.get_nearest_node(G, start) 
        end_node = ox.get_nearest_node(G, end)
        routes.append([vehicle,nx.shortest_path(G, start_node, end_node, weight='travel_time')])

In [ ]:
import pandas as pd

node_start = {vehicle: [] for vehicle in vehicles}
node_end = {vehicle: [] for vehicle in vehicles}
X_to = {vehicle: [] for vehicle in vehicles}
Y_to = {vehicle: [] for vehicle in vehicles}
X_from = {vehicle: [] for vehicle in vehicles}
Y_from = {vehicle: [] for vehicle in vehicles}
length = {vehicle: [] for vehicle in vehicles}
travel_time = {vehicle: [] for vehicle in vehicles}

for track in routes:
    for u, v in zip(track[1][:-1], track[1][1:]):
        node_start[track[0]].append(u)
        node_end[track[0]].append(v)
        length[track[0]].append(round(G.edges[(u, v, 0)]['length']))
        travel_time[track[0]].append(round(G.edges[(u, v, 0)]['travel_time']))
        X_from[track[0]].append(G.nodes[u]['x'])
        Y_from[track[0]].append(G.nodes[u]['y'])
        X_to[track[0]].append(G.nodes[v]['x'])
        Y_to[track[0]].append(G.nodes[v]['y'])

routes_dfs = {vehicle : pd.DataFrame(list(zip(node_start[vehicle], node_end[vehicle], X_from[vehicle], Y_from[vehicle],
               X_to[vehicle], Y_to[vehicle], length[vehicle], travel_time[vehicle])), 
               columns =['node_start', 'node_end', 'X_from', 'Y_from',  'X_to',
               'Y_to', 'length', 'travel_time']) for vehicle in vehicles}